<a href="https://colab.research.google.com/github/ronykroy/NLP/blob/master/text_summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Summarization : the process of creating short and meaningful summaries from a larger text [duh]  
**Extractive Methods**: These methods rely on extracting several parts, such as phrases and sentences, from a piece of text and stack them together to create a summary. Therefore, identifying the right sentences for summarization is of utmost importance in an extractive method.   
1. TextRank
2. LexRank
3. LSA

**Abstractive Methods**: 
1. Encoder Decoder
2. seq2seq RNN
3. Transformer bert

**Combination Approach**:
1. Extract then Abstract model


In [0]:
# data https://archive.ics.uci.edu/ml/datasets/Legal+Case+Reports
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00239/corpus.zip
# https://www.affineanalytics.com/how-to-build-a-legal-document-summarizer/

--2019-11-05 11:47:40--  https://archive.ics.uci.edu/ml/machine-learning-databases/00239/corpus.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84720359 (81M) [application/x-httpd-php]
Saving to: ‘corpus.zip.1’

corpus.zip.1        100%[===================>]  80.79M  36.2MB/s    in 2.2s    

2019-11-05 11:47:42 (36.2 MB/s) - ‘corpus.zip.1’ saved [84720359/84720359]



In [0]:
!unzip corpus.zip

Archive:  corpus.zip
replace corpus/citations_class/06_1.xml? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [0]:
ls corpus

citations_class/  citations_summ/  fulltext/  readme.txt


In [0]:
ls corpus/citations_class/ | wc -l

2754


In [0]:
ls corpus/citations_summ/ | wc -l

3890


In [0]:
ls corpus/fulltext/ | wc -l

3890


In [0]:
import os,sys

In [0]:
import spacy
!python -m spacy download en_core_web_lg # big model

In [0]:
import re
import pandas as pd
import spacy
nlp = spacy.load('en_core_web_lg')

In [0]:
def read_and_add_text_fromXML_inPath(path):
  l_sum = os.listdir(path)
  list_return = [['file','text']]
  for file_name in l_sum:
    f = open(path+file_name, "r",encoding="latin-1") # silly me
    cleaned_text = [re.sub('<.+?>',' ',w) for w in f.readlines()]
    cleaned_text = [re.sub('&amp;','',w) for w in cleaned_text]
    cleaned_text = [w.rstrip("\n\r") for w in cleaned_text]
    cleaned_text = [re.sub('\s+?',' ',w) for w in cleaned_text]
    text = ''.join(cleaned_text)
    list_temp = [file_name,text]
    list_return.append(list_temp)
  return list_return

In [0]:
list_citations_summary = read_and_add_text_fromXML_inPath('corpus/citations_summ/')

In [0]:
df_summ = pd.DataFrame(list_citations_summary)

In [0]:
list_citations_fulltext = read_and_add_text_fromXML_inPath('corpus/fulltext/')

In [0]:
df_full = pd.DataFrame(list_citations_fulltext)

In [11]:
df_summ = df_summ[1:];df_summ.columns= ['fileName','summary'];df_summ

,fileName,summary
1,06_1362.xml,Rahman v Dayeh [2006] FCA 1362 (11 October ...
2,07_301.xml,Military Rehabilitation and Compensation Co...
3,08_904.xml,SZLAN v Minister for Immigration and Citize...
4,06_1803.xml,SZICO v Minister for Immigration and Multic...
5,08_1960.xml,33 South Pty Ltd v Fitzgerald [2008] FCA 19...
...,...,...
3886,07_1705.xml,SZKET v Minister for Immigration Citizensh...
3887,07_2089.xml,Haslam v Money for Living (No 3) [2007] FCA...
3888,06_1046.xml,Nine Films Television Pty Limited v Ninox ...
3889,08_702.xml,Sterling Commerce (Australia) Pty Ltd v Ili...


In [12]:
df_full = df_full[1:]; df_full.columns= ['fileName','text'];df_full

,fileName,text
1,06_1362.xml,Rahman v Dayeh [2006] FCA 1362 (11 October ...
2,07_301.xml,Military Rehabilitation and Compensation Co...
3,08_904.xml,SZLAN v Minister for Immigration and Citize...
4,06_1803.xml,SZICO v Minister for Immigration and Multic...
5,08_1960.xml,33 South Pty Ltd v Fitzgerald [2008] FCA 19...
...,...,...
3886,07_1705.xml,SZKET v Minister for Immigration Citizensh...
3887,07_2089.xml,Haslam v Money for Living (No 3) [2007] FCA...
3888,06_1046.xml,Nine Films Television Pty Limited v Ninox ...
3889,08_702.xml,Sterling Commerce (Australia) Pty Ltd v Ili...


In [0]:
df_combined = pd.merge(df_full, df_summ, on='fileName', how='outer')

In [14]:
df_combined

,fileName,text,summary
0,06_1362.xml,Rahman v Dayeh [2006] FCA 1362 (11 October ...,Rahman v Dayeh [2006] FCA 1362 (11 October ...
1,07_301.xml,Military Rehabilitation and Compensation Co...,Military Rehabilitation and Compensation Co...
2,08_904.xml,SZLAN v Minister for Immigration and Citize...,SZLAN v Minister for Immigration and Citize...
3,06_1803.xml,SZICO v Minister for Immigration and Multic...,SZICO v Minister for Immigration and Multic...
4,08_1960.xml,33 South Pty Ltd v Fitzgerald [2008] FCA 19...,33 South Pty Ltd v Fitzgerald [2008] FCA 19...
...,...,...,...
3885,07_1705.xml,SZKET v Minister for Immigration Citizensh...,SZKET v Minister for Immigration Citizensh...
3886,07_2089.xml,Haslam v Money for Living (No 3) [2007] FCA...,Haslam v Money for Living (No 3) [2007] FCA...
3887,06_1046.xml,Nine Films Television Pty Limited v Ninox ...,Nine Films Television Pty Limited v Ninox ...
3888,08_702.xml,Sterling Commerce (Australia) Pty Ltd v Ili...,Sterling Commerce (Australia) Pty Ltd v Ili...


1. Remove punctuations and special characters from the text
2. Convert all the alphabets into lower case
3. Remove STOPWORDS
4. Do not remove numbers and dates, they can be very important for a legal case

In [0]:
df_combined["summary"]= df_combined["summary"].str.replace("[^a-zA-z0-9]", " ") 
df_combined["text"]= df_combined["text"].str.replace("[^a-zA-z0-9]", " ") # apply tot both..?

In [0]:
clean_text = df_combined["summary"].str.replace("[^a-zA-z0-9\.]", " ") 

In [0]:
clean_text = [s.lower() for s in clean_text]

In [17]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [0]:
from nltk.corpus import stopwords

In [0]:
stopwords = set(stopwords.words('english'))

In [0]:
#stopwords

In [21]:
 " ".join([i for i in  clean_text[10].split() if i not in stopwords])

'moloney v new zealand 2 [2006] fca 603 1 may 2006 http www austlii edu au au cases cth fca 2006 603 html extradition overseas new zealand eligibility application review old allegations sexual offences vulnerable children alleged members religious order whether age charges circumstances made unjust oppressive surrender review magistrate decision differences australian new zealand criminal law procedure principle bannister v new zealand 1999 86 fcr 417 costs magistrates courts information dismissal discretion award costs defendant relevant considerations magistrates summary proceedings act 1975 vict 97 b extradition act 1988 federal court australia act 1976 federal court australia act 1976 sect 43 costs'

In [0]:
def remove_stopwords(txt):
  return " ".join([i for i in txt if i not in stopwords ])

In [0]:
clean_text_stopped = [remove_stopwords(r.split()) for r in clean_text]

In [51]:
clean_text [11]

'   qaai v minister for immigration and multicultural and indigenous affairs [2006] fca 4  10 january 2006   http   www.austlii.edu.au au cases cth fca 2006 4.html   applicant veal of 2002 v minister for immigration and multicultural and indigenous affairs  immigration  application for protection visa  decision of refugee review tribunal  procedural fairness  where tribunal reviewed an unsolicited letter received by the department of immigration and multicultural and indigenous affairs  which made allegations against the appellant  where tribunal did not inform the appellant of the existence of the letter or its contents  where tribunal affirmed decision under review and said that in reaching its decision it gave no weight to the letter  whether procedural fairness required the tribunal to inform the appellant of the existence of the letter or its contents.  minister for immigration and multicultural and indigenous affairs v sglb  immigration  refugees  procedural fairness  refugee rev

In [25]:
clean_text_stopped[11]

'qaai v minister immigration multicultural indigenous affairs [2006] fca 4 10 january 2006 http www austlii edu au au cases cth fca 2006 4 html applicant veal 2002 v minister immigration multicultural indigenous affairs immigration application protection visa decision refugee review tribunal procedural fairness tribunal reviewed unsolicited letter received department immigration multicultural indigenous affairs made allegations appellant tribunal inform appellant existence letter contents tribunal affirmed decision review said reaching decision gave weight letter whether procedural fairness required tribunal inform appellant existence letter contents minister immigration multicultural indigenous affairs v sglb immigration refugees procedural fairness refugee review tribunal jurisdictional error expert medical advice psychological state applicant assumed possible explanation unsatisfactory evidence whether denial procedural fairness failing order psychological assessment applicant wheth

In [0]:
nlp_lm = spacy.load('en_core_web_lg')

In [0]:
text = clean_text_stopped[1]# save point kinda

In [0]:
doc = nlp_lm(text)

In [36]:
counter =0
for entity in doc.ents:
  if counter >8:
    break
  print(f'{entity.text} :: {entity.label_}')
  counter +=1

2007 :: DATE
301 13 march :: TIME
2007 :: DATE
1989 :: DATE
62 :: CARDINAL
169 :: CARDINAL
379 :: CARDINAL
period years :: DATE
1936 :: DATE


In [0]:
import gensim

In [0]:
from gensim.summarization.summarizer import summarize

In [46]:
clean_text_stopped[2]

'szlan v minister immigration citizenship [2008] fca 904 13 june 2008 http www austlii edu au au cases cth fca 2008 904 html migration decision upholding refusal protection class xa visa whether tribunal entitled base finding demeanour tribunal entitled regard demeanour tribunal rejection claims based several strands reasoning one demeanour whether tribunal obliged ask questions tribunal obliged frame questions findings adverse appellant indicate bias apprehended bias jurisdictional error migration test well founded fear persecution whether tribunal applied wrong test probability test whether real chance test chan v minister immigration ethnic affairs [1989] hca 62 1989 169 clr 379 applied past persecutory event claimed findings past events category assessment future events tribunal reasons findings past events attended doubt tribunal required consider past event applying real chance test migration review decision refugee review tribunal whether refugee review tribunal decision affecte

In [52]:
summarize( clean_text_stopped[2], word_count = 100)

'effect s.36 3 applicant fails demonstrate taken reasonable steps exercise legally enforceable entry residency rights may another country australia taken protection obligations person minister immigration multicultural affairs v applicant c [2001] fca 1332 2001 116 fcr 154 szlan v minister immigration citizenship [2008] fca 904 2008 171 fcr 145 159 [58] .'

In [44]:
# terrible actually

''